## 1. Library Import

In [2]:
import numpy as np
import time
import itertools

__author__ = "Marco Odehnal"
__copyright__ = "Copyright 2018"
__status__ = "Prototype"

## 2. Problem formulation

In [183]:
# The cities for the problem, taken from att48
cities = np.matrix([[1, 6734, 1453],
                    [2, 2233, 10],
                    [3, 5530, 1424],
                    [4, 401, 841],
                    [5, 3082, 1644],
                    [6, 7608, 4458],
                    [7, 7573, 3716],
                   [8, 123, 8526],
                   [9, 4356, 376],
                   [10, 7456, 236],
                    [11, 67534, 14553],
                    [2, 22353, 150],
                    [3, 55530, 14524],
                    [4, 45501, 841],
                    [5, 350852, 16544],
                    [6, 76508, 4458],
                    [7, 75573, 35716],
                   [8, 1253, 85526],
                   [9, 43556, 5376],
                   [10, 57456, 2536]                   ])

# The number of verteces 
n = np.size(cities,0)

# Adjacency matrix
# NOT OPTIMAL GENERATION, JUST FOR TESTING
M = np.matrix([[np.linalg.norm(cities[i,1:3]-cities[j,1:3]) for j in range(n)] for i in range(n)])

## 3. Classic approach

In [138]:
M = np.matrix([[np.inf,2,3,5],
             [2,np.inf,6,1],
             [3,6,np.inf,4],
             [5,1,4,np.inf]])

rem_nodes = list(range(2,4+1))
cost = 0
optimal_cost = np.inf
v = [1]

# Note: the third variable optimal cost is used to determine if we have reached the end of a branch

def BnBClassic(A,v,rem_nodes, cost, optimal_cost):
    
    # End of the path
    if len(rem_nodes)==1:
        cost += A[v[-1]-1, rem_nodes[0]-1] + A[rem_nodes[0]-1,0]
        v += rem_nodes + [1]
        return cost, v, cost
    
    else:
        
        k = len(v)
        n = np.size(A,0)

        cost_branches = []
        path_branches = []
        f = []

        # Calculating the costs, the f and the paths
        for i in rem_nodes:
            cost_branches.append(cost + A[v[-1]-1,i-1])
            path_branches.append(v+[i])
            f.append(cost_branches[-1] + (n-k)*
                     np.min(A[path_branches[-1][-1]-1,[x-1 for x in rem_nodes if x!= k]]))

        # Sorting the arrays
        order = np.argsort(f)
        cost_branches = [cost_branches[i] for i in order]
        path_branches = [path_branches[i] for i in order]
        f = [f[i] for i in order]      


        # We explore recursively the branches and check if an optimal solution can be found
        for i in range(len(f)):
            # We discard all of the branches that cannot decrease the cost function
            # As all of the branches are sorted by cost, the following branches after
            # a discarded one will also be discarded
            if f[i] >= optimal_cost:
                break
            else:
                rem_nodes_sub = [x for x in rem_nodes if x not in path_branches[i]]
                cost, v, optimal_cost = BnBClassic(A,path_branches[i],rem_nodes_sub, cost_branches[i], optimal_cost)
            
            return cost, v, optimal_cost

t1 = time.time()     
    
cost, path, opt = BnBClassic(M,v,rem_nodes,cost,optimal_cost)

t2= time.time()

print('Time:', t2-t1)    
print('Minimum_cost:',cost)
print('Best path:',path)    

Time: 0.0010008811950683594
Minimum_cost: 10.0
Best path: [1, 2, 4, 3, 1]


## 4. Adding and removing edges

# DISREGARD THIS SECTION, UNDER CONSTRUCTION

In [185]:
# M = np.matrix([[np.inf,2,3,5],
#              [2,np.inf,6,1],
#              [3,6,np.inf,4],
#              [5,1,4,np.inf]])

# print(M)
L = list(range(2,10+1))

all_S = []
for i in range(1,len(L)+1):
    all_S = all_S + list(itertools.combinations(L, i))

# print(all_S)    

# start algorithm
cost_memo = {}
path_memo = {}

t1 = time.time()

#We compute the edges from the leaves to the starting point
for k in all_S[0:len(L)]:
   cost_memo[k] = M[k[0]-1,0]
   path_memo[k] = [k[0]]
    
# We traverse through all the list of subsets
for k in all_S[len(L):]:
    
    # Initialize minimal cost
    min_cost = np.inf 
    
    # We traverse through the elements of the permutation
    for i in range(0,len(k)): # This index removes the element corresponding to the node that we will visit
        for j in range(0,len(k)): # This index corresponds to the current node, i != j
                                  # E.G. [k={2,3,4}, i=3, j=2] ==> [2 -> 3, with j in k-{3}] 
            if i==j: continue
            
            # We remove "i" from the subset, but tuples are immutable
            s = list(k)
            s.remove(k[i])
            s = tuple(s)
            
            # The new cost is the sum of the cost of traveling to i + accumulated cost
            cost = M[k[i]-1,k[j]-1] + cost_memo[s]
            
            # We look for minimum cost and its path
            if cost < min_cost:
                min_cost = cost
                min_path = path_memo[s] + [k[i]]
                
    # We save in our memos the cost and its associated path
    cost_memo[k] = min_cost    
    path_memo[k] = min_path

# We add the set corresponding to (1,2,...,n) to our calculations
full_set = tuple(range(1,len(L)+2))
path_memo[full_set] = [1] + min_path
cost_memo[full_set] = min_cost + M[0,min_path[-1]-1]
    
t2= time.time()
print('Time:', t2-t1)    
print('Minimum_cost:',cost_memo[full_set])
print('Best path:',path_memo[full_set])
    

Time: 0.03702592849731445
Minimum_cost: 28358.3769423
Best path: [1, 3, 9, 5, 8, 2, 7, 10, 6, 4]


In [145]:
print(path_memo[(2,3,4,5,6)])

[4, 5, 6, 3, 2]
